In [6]:
%run -i "../util/file_utils.ipynb"
%run -i "../util/lang_utils.ipynb"
%run -i "../util/util_simple_classifier.ipynb"

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embedding = model.encode(["I love jazz"])
print(embedding)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

[[ 2.94216443e-03 -7.93536380e-02 -2.82228086e-02 -5.13780564e-02
  -6.44980818e-02  9.83557850e-02  1.09671935e-01 -3.26390453e-02
   4.96566594e-02  2.56580524e-02 -1.08482078e-01  1.88441798e-02
   2.70963982e-02 -3.80690508e-02  2.42502466e-02 -3.65601829e-03
   1.29364476e-01  4.32254896e-02 -6.64561763e-02 -6.93060607e-02
  -1.39410645e-01  4.36719991e-02 -7.85465166e-03  1.68625209e-02
  -1.01159597e-02  1.07926447e-02 -1.05814300e-02  2.57284790e-02
  -1.51516097e-02 -4.53921035e-02  7.12088868e-03  1.17573030e-01
  -1.31496182e-03 -2.59251427e-02 -8.34854990e-02  3.65455300e-02
  -2.86979452e-02  3.15047912e-02  3.97394188e-02  5.78807108e-02
  -2.24545319e-02  7.98929557e-02  4.24232241e-03 -1.34443417e-02
  -7.81929493e-02 -1.82035677e-02  6.06971746e-03 -3.64662781e-02
   8.80980790e-02  5.81041239e-02 -3.77037749e-03  2.63834018e-02
   4.09417748e-02  5.37395924e-02 -1.32099669e-02  2.30073798e-02
  -5.78504913e-02  9.60877314e-02  1.13227377e-02 -4.31320816e-02
   7.90958

In [3]:
def get_sentence_vector(text, model):
    sentence_embeddings = model.encode([text])
    return sentence_embeddings[0]

In [9]:
import time
vectorize = lambda x: get_sentence_vector(x, model)
(train_df, test_df) = load_train_test_dataset_pd()
start = time.time()
(X_train, X_test, y_train, y_test) = create_train_test_data(train_df, test_df, vectorize)
print(f"BERT embedding vectorization time: {time.time() - start} seconds")


BERT embedding vectorization time: 46.86356163024902 seconds


In [10]:
clf = train_classifier(X_train, y_train)
test_classifier(test_df, clf)

              precision    recall  f1-score   support

           0       0.77      0.79      0.78       160
           1       0.79      0.76      0.77       160

    accuracy                           0.78       320
   macro avg       0.78      0.78      0.78       320
weighted avg       0.78      0.78      0.78       320



In [11]:
from openai import OpenAI
client = OpenAI()
model = "text-embedding-ada-002"
text = "I love jazz"
response = client.embeddings.create(input=text, model=model)
embedding = response.data[0].embedding
print(embedding)

[-0.028504954650998116, -0.011232884600758553, -0.0023371647112071514, -0.014288712292909622, -0.012096487917006016, 0.01834704540669918, -0.010236418806016445, -0.017924301326274872, -0.014228320680558681, 0.0008492604247294366, -0.0001670212222961709, 0.010176027193665504, 0.011498608626425266, -0.014131693169474602, -0.007253061514347792, -0.01849198527634144, 0.03857831656932831, 0.002498712856322527, 0.03466492518782616, -0.02666904218494892, -0.011063787154853344, -0.0036114328540861607, 0.018793944269418716, -0.023516587913036346, 0.010562535375356674, 0.020968051627278328, -0.0063351052813231945, -0.005157464183866978, -0.02326294220983982, -0.003822804195806384, 0.048047758638858795, 0.012549427337944508, -0.03640420734882355, -0.03669409081339836, -0.013165423646569252, -0.024277525022625923, -0.02105259895324707, -0.0172599907964468, -0.0015369727043434978, -0.008654152043163776, 0.001001749886199832, 0.020062172785401344, -0.006721612997353077, 0.004227429628372192, -0.0086

In [12]:
def get_sentence_vector(text, model):
    response = client.embeddings.create(input=text, model=model)
    embedding = response.data[0].embedding
    return embedding

In [13]:
import time
vectorize = lambda x: get_sentence_vector(x, model)
(train_df, test_df) = load_train_test_dataset_pd()
start = time.time()
(X_train, X_test, y_train, y_test) = create_train_test_data(train_df, test_df, vectorize)
print(f"OpenAI embedding vectorization time: {time.time() - start} seconds")

OpenAI embedding vectorization time: 604.2199828624725 seconds


In [14]:
clf = train_classifier(X_train, y_train)
test_classifier(test_df, clf)

              precision    recall  f1-score   support

           0       0.87      0.86      0.87       160
           1       0.86      0.88      0.87       160

    accuracy                           0.87       320
   macro avg       0.87      0.87      0.87       320
weighted avg       0.87      0.87      0.87       320



In [16]:
# RAG
import csv
import openai
from llama_index.core import VectorStoreIndex, Document

In [18]:
with open('../data/IMDB-Movie-Data.csv') as f:
    reader = csv.reader(f)
    data = list(reader)
    movies = data[1:]  # Skip header row

In [19]:
documents = []
for movie in movies[0:10]:
    doc_id = movie[0]
    title = movie[1]
    genres = movie[2].split(",")
    description = movie[3]
    director = movie[4]
    actors = movie[5].split(",")
    year = movie[6]
    duration = movie[7]
    rating = movie[8]
    revenue = movie[10]
    document = Document(
        text=description, 
        metadata={
            "title": title, 
            "genres": genres, 
            "director": director, 
            "actors": actors, 
            "year": year, 
            "duration": duration, 
            "rating": rating, 
            "revenue": revenue
        }
    )
    print(document)
    documents.append(document)
index = VectorStoreIndex.from_documents(documents)

Doc ID: a379913c-cc12-42c4-b621-524873af3198
Text: A group of intergalactic criminals are forced to work together
to stop a fanatical warrior from taking control of the universe.
Doc ID: ae8a1f36-1e9c-4526-85c0-3a7aa731148a
Text: Following clues to the origin of mankind, a team finds a
structure on a distant moon, but they soon realize they are not alone.
Doc ID: a820fed4-2095-437d-83a8-4d46257795dc
Text: Three girls are kidnapped by a man with a diagnosed 23 distinct
personalities. They must try to escape before the apparent emergence
of a frightful new 24th.
Doc ID: 5a7af46b-05c4-4598-abb5-47627deeb355
Text: In a city of humanoid animals, a hustling theater impresario's
attempt to save his theater with a singing competition becomes grander
than he anticipates even as its finalists' find that their lives will
never be the same.
Doc ID: ac5eb2bc-7254-4451-862a-4fd73cd42f45
Text: A secret government agency recruits some of the most dangerous
incarcerated super-villains to form a defensi

In [21]:
query_engine = index.as_query_engine()
response = query_engine.query("Which movies talk about something gigantic?")
print(response)

"The Great Wall" talks about something gigantic.
